In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import torch

In [205]:
df = pd.read_csv('output_data/ucf_lat_lon_interpolated_all_chars.csv')

In [206]:
bins = [-6.125, -5.126, -4.126, -3.126, -2.126, -1.126, -0.126, 0.874, 3.874, 4.874, 5.874,\
6.874, 7.874, 8.874, 9.874, 11.874, 12.874, 13.874, 14.874, 15.874, 16.874, 17.875]

In [207]:
df['label']=pd.cut(df['pred_post_pct_change'], bins, labels = range(1, len(bins)))

In [208]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 

X = ['pred_density2019', 'pred_wfh_emp', 'pred_dist_to_cbd', 'pred_estab_count']
y = 'label'

#scaler = StandardScaler()
#dfx = pd.DataFrame(scaler.fit_transform(df[X]), columns = X)
df_clean = df[X+[y]]
df_clean = sm.add_constant(df_clean)
df_clean=df_clean.dropna()

X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.33)

# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)
preds = model.predict(X_test)
class_report=classification_report(y_test, preds)
print(sum(y_test==preds)/len(y_test))
print(class_report)

/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


0.4431751611013474
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        57
           2       0.53      0.77      0.63       354
           3       0.40      0.38      0.39       437
           4       0.10      0.05      0.07       321
           5       0.20      0.09      0.12       258
           6       0.56      0.89      0.68       392
           7       0.00      0.00      0.00        51
           8       0.45      0.73      0.56        73
           9       0.00      0.00      0.00        76
          10       0.72      1.00      0.84       306
          11       0.00      0.00      0.00        43
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00        51
          15       0.00      0.00      0.00       284
          16       0.00      0.00      0.00       262
          17       0.33      1.00      0.50       328
          18       0.00      0.00      0.00        92
        

/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1

In [209]:
np.sqrt(mean_squared_error(y_run, result.predict(X_run), squared=False))

0.9487686692155199

Read in image data to evaluate on model

In [291]:
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm 
from main import make_weights_for_balanced_classes, measure_accuracy, ImageClassificationCollator
from models import LogisticRegression #, BasicCNNModel, DenseCNNModel
from torchvision import transforms
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from transformers import ViTFeatureExtractor, ViTForImageClassification


In [304]:
def measure_accuracy(outputs, labels, CLASS_NAMES):
	preds = np.argmax(outputs, axis = 1).flatten()
	labels = labels.flatten()
	correct = np.sum(preds == labels)
	big_c_matrix = confusion_matrix(labels, preds, labels=CLASS_NAMES)
	c_matrices = multilabel_confusion_matrix(labels, preds, labels=CLASS_NAMES)
	TNs=0; FPs=0; FNs=0; TPs=0;
	for c_matrix in c_matrices:	
		TN, FP, FN, TP = c_matrix.ravel()
		TNs+=TN; FPs+=FP; FNs+=FN; TPs+=TP
	correct = np.sum(labels==preds)
	return TNs, FPs, FNs, TPs, correct, len(labels), big_c_matrix 

In [305]:
CLASSES=20
CLASS_NAMES = [i for i in range(CLASSES)]

In [306]:
## read in logistic_regression1.pt
model = ViTForImageClassification.from_pretrained('models/vit/')
model.cuda()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_fea

In [307]:
def test(loader, model, criterion, device, name="test", write_file=None):
	#need criterion, device
	model.eval()
	total_loss = 0. 

	total_TN=0; total_FP=0; total_FN=0; total_TP=0
	total_correct=0
	total_sample = 0 
	total_confusion = np.zeros((CLASSES, CLASSES))

	predicted_labels = []

	for i, batch in enumerate(tqdm(loader)):
		inputs, labels = batch['pixel_values'], batch['labels'] 
		inputs = inputs.to(device)
		labels = labels.to(device)

		with torch.no_grad(): logits = model(inputs)['logits']
		
		predicted_labels.append(logits)

		logits = logits.to(device)
		loss = criterion(logits, labels)
		total_loss += loss.cpu().item()

		TN, FP, FN, TP, n_correct, sample_n, c_matrix  = measure_accuracy(logits.cpu().numpy(), labels.cpu().numpy(), CLASS_NAMES)
		total_correct+=n_correct; total_sample+=sample_n; total_confusion+=c_matrix
		
		total_TN+=TN; total_FP+=FP; total_FN+=FN; total_TP+=TP

	print(f'*** Accuracy on the {name} set: {total_correct/total_sample}')
	print(f'*** Precision on the {name} set: {TP/(TP+FP)}')
	print(f'*** Recall on the {name} set: {TP/(TP+FN)}')
	#print(f'*** Confusion matrix:\n{total_confusion}')
	if write_file:
		write_file.write(f'*** Accuracy on the {name} set: {total_correct/total_sample}\n')
		write_file.write(f'*** Confusion matrix:\n{total_confusion}\n')

	test_acc = float(total_correct / total_sample) * 100

	return total_loss, test_acc, predicted_labels

In [308]:
dataset = ImageFolder(root='main_data2_east_val') #lazily loads all images

In [309]:
weights, class_weights = make_weights_for_balanced_classes(dataset.imgs, len(dataset.classes))
weights = torch.FloatTensor(weights)
class_weights = torch.FloatTensor(class_weights)
criterion = nn.CrossEntropyLoss(weight = class_weights).cuda()

In [310]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
collator = ImageClassificationCollator(feature_extractor)

test_dataset = DataLoader(dataset, batch_size=32, collate_fn=collator)

In [311]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [312]:
_, _, predicted_labels = test(loader=test_dataset, model=model, criterion=criterion, device=device)

100%|██████████| 45/45 [00:48<00:00,  1.08s/it]

*** Accuracy on the test set: 0.2523033309709426
*** Precision on the test set: 0.3333333333333333
*** Recall on the test set: 0.3333333333333333


In [219]:
predicted_labels2 = [predicted_labels[i].cpu().numpy() for i in range(len(predicted_labels))]

In [220]:
predicted_labels3 = np.array([x for y in predicted_labels2 for x in y])

In [221]:
predicted_labels3 = pd.DataFrame(predicted_labels3.reshape((-1, 20)))

In [222]:
predicted_labels3.columns=['label_' + str(i) for i in predicted_labels3]

In [223]:
paths, _=zip(*dataset.imgs)
new_paths = [path.split('/')[2] for path in paths]
indices = [int(f.split('_')[0]) for f in new_paths]

In [224]:
predicted_labels3['merge_col']=pd.Series(indices)

In [225]:
labels_df = predicted_labels3

In [226]:
labels_df

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,...,label_11,label_12,label_13,label_14,label_15,label_16,label_17,label_18,label_19,merge_col
0,-0.529058,-0.672272,-0.828284,-1.047476,3.657719,2.420945,2.969915,-0.778145,-1.693525,-1.175701,...,-0.031584,0.180130,1.771587,5.214890,2.803714,-0.328333,-1.008610,-0.707124,-1.374020,4408
1,-0.177726,0.171371,-0.714940,-1.661297,3.356462,2.060072,2.660975,-1.033036,-0.853733,-1.154686,...,-0.686790,0.903336,1.941999,6.253850,1.565685,-0.524279,-0.704502,-0.715605,-0.720212,4422
2,-0.421651,0.621193,-0.875519,-1.583046,1.252312,1.149874,1.612896,-0.394665,0.054276,-0.966672,...,-1.065666,1.338967,2.495767,7.124341,2.187498,-0.276057,-0.847643,-0.310195,-0.693611,4423
3,-0.672412,-0.482491,-0.868200,-1.024541,2.039017,0.803414,1.053237,-1.229863,-2.069312,-1.152663,...,-0.453309,0.711398,3.793168,6.514907,3.794643,-0.494839,-0.547396,-0.426675,-1.486368,4426
4,-0.430698,0.628553,-0.710090,-1.138537,0.848180,-0.055030,0.373838,-0.970629,-0.720325,-1.063880,...,-0.681198,1.840883,2.906957,6.714512,3.419033,0.100626,0.092384,0.499582,-0.830592,6741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,-0.203982,0.243386,0.296892,-1.376281,-0.436716,-0.730272,-0.391213,-0.189076,-0.089640,-0.601886,...,-0.894859,0.640258,-0.797403,0.212239,-0.958571,-0.962498,-0.937307,2.432998,7.194004,3497
1407,-0.877032,-0.414930,-0.135615,-1.288178,-0.969486,-1.360254,0.356759,0.842337,-0.540720,-0.575196,...,-0.347059,0.261281,-0.478108,0.020949,1.239966,-0.628931,2.170654,7.449111,2.317293,3596
1408,-0.370197,-0.576587,0.524093,-1.077670,-1.387510,-1.432783,-0.635655,0.040635,-0.320703,0.393489,...,-1.019457,-0.071306,-0.066461,-1.191395,-0.076136,-1.358922,4.308955,7.073775,1.445572,3603
1409,-1.164890,-1.006400,0.782440,0.678122,-0.773088,-0.846484,-0.975124,0.449184,-0.813567,-1.093453,...,-0.997444,0.172792,0.436217,-0.353437,-0.739387,-1.074814,-0.806125,2.493350,7.284929,4038


In [227]:
df=df.merge(labels_df, on='merge_col')

In [228]:
df

,Unnamed: 0,lat,lon,compass,merge_col,zip,post_pct_change,pre_pct_change,density2019,wfh_emp,...,label_10,label_11,label_12,label_13,label_14,label_15,label_16,label_17,label_18,label_19
0,1,40.440271,-80.006800,119.24,2,15211,18.814366,8.243265,6657.455182,0.401119,...,-0.746721,-0.164411,0.488373,0.565465,3.764569,0.315173,-1.478644,-1.884411,-0.059268,-0.604174
1,3,40.440189,-80.006600,118.68,4,15211,18.814366,8.243265,6657.455182,0.401119,...,-0.072316,-1.151779,1.567367,2.636570,6.455995,1.817873,-0.359595,-0.913241,-0.814075,-0.842576
2,13,40.439720,-80.005300,115.87,14,15211,18.814366,8.243265,6657.455182,0.401119,...,-0.948428,-1.077294,-0.045520,0.792635,2.127646,0.494998,-0.407948,-2.152733,-1.069296,-1.174233
3,17,40.439475,-80.004000,114.74,18,15222,8.064086,2.621717,6583.614873,0.484400,...,-0.844793,-1.871219,-1.168997,-1.181048,-0.943008,-1.862635,-1.224407,-2.236741,-2.171866,-1.151943
4,19,40.439381,-80.004400,114.18,20,15222,8.064086,2.621717,6583.614873,0.484400,...,-0.250169,-0.985914,-0.036094,0.902289,2.437139,-0.350065,-1.385200,-2.140086,-1.411850,-1.419021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,8772,40.729771,-73.986812,143.92,8773,10003,-0.508407,-3.709946,93637.009680,0.505173,...,-0.515669,-0.691545,3.720292,4.842623,6.361959,4.914660,-0.075143,-0.564232,0.270296,-0.444728
1407,8776,40.729586,-73.986371,158.48,8777,10003,-0.508407,-3.709946,93637.009680,0.505173,...,-0.390951,-0.482745,2.829035,4.687967,6.888363,4.276443,-0.086221,-0.867466,-0.481046,-0.613991
1408,8781,40.729330,-73.985757,176.68,8782,10003,-0.508407,-3.709946,93637.009680,0.505173,...,-0.373208,-0.649045,1.636981,4.776500,6.247315,5.745951,0.135550,-0.537219,0.083764,-0.906110
1409,8784,40.729164,-73.985359,187.60,8785,10003,-0.508407,-3.709946,93637.009680,0.505173,...,-0.312394,-0.364262,0.854075,3.117109,4.975822,6.986619,0.635137,-0.848219,-0.544822,-1.117901


In [229]:
new_X = ['label_' + str(i) for i in range(20)]

In [239]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 

X = ['pred_density2019', 'pred_wfh_emp', 'pred_dist_to_cbd', 'pred_estab_count']+new_X
y = 'label'

#scaler = StandardScaler()
#dfx = pd.DataFrame(scaler.fit_transform(df[X]), columns = X)
df_clean = df[X+[y]]
df_clean = sm.add_constant(df_clean)
df_clean=df_clean.dropna()

X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.33, random_state=30)

# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)
preds = model.predict(X_test)
class_report=classification_report(y_test, preds)
print(sum(y_test==preds)/len(y_test))
print(class_report)

/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


0.5107296137339056
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        12
           2       0.59      0.89      0.71        56
           3       0.42      0.28      0.33        47
           4       0.58      0.45      0.51        42
           5       0.45      0.68      0.54        31
           6       0.68      0.63      0.66        41
           7       0.00      0.00      0.00         6
           9       0.00      0.00      0.00        12
          10       0.75      1.00      0.86        54
          11       0.00      0.00      0.00         6
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         8
          15       0.47      0.20      0.28        40
          16       0.06      0.05      0.06        39
          17       0.41      1.00      0.58        45
          18       0.00      0.00      0.00        16
          19       0.00      0.00      0.00         4
        

/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1

In [240]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 

X = ['pred_density2019', 'pred_wfh_emp', 'pred_dist_to_cbd', 'pred_estab_count']
y = 'label'

#scaler = StandardScaler()
#dfx = pd.DataFrame(scaler.fit_transform(df[X]), columns = X)
df_clean = df[X+[y]]
df_clean = sm.add_constant(df_clean)
df_clean=df_clean.dropna()

X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.33, random_state=30)

# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)
preds = model.predict(X_test)
class_report=classification_report(y_test, preds)
print(sum(y_test==preds)/len(y_test))
print(class_report)

0.5236051502145923
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        12
           2       0.60      0.79      0.68        56
           3       0.41      0.38      0.40        47
           4       0.61      0.40      0.49        42
           5       0.43      0.68      0.53        31
           6       0.68      0.68      0.68        41
           7       0.00      0.00      0.00         6
           9       0.00      0.00      0.00        12
          10       0.75      1.00      0.86        54
          11       0.00      0.00      0.00         6
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         8
          15       0.47      0.20      0.28        40
          16       0.38      0.95      0.54        39
          17       0.38      0.38      0.38        45
          18       0.00      0.00      0.00        16
          19       0.00      0.00      0.00         4
        

/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0